This notebook is innovated from [Analysis of Melanoma Metadata and EffNet Ensemble
](https://www.kaggle.com/datafan07/analysis-of-melanoma-metadata-and-effnet-ensemble/data?), greate work and thanks!

The reason to write this is that I've already tuned my deep learning models for both tensorflow(wihout tabular data) and pytorch(with tabualr data). When I want more essence from tabular data I see this greate notebook from Ertuğrul Demir (Thanks again!). So I extract the part of XGBoost from the notebook and do some refactor for fine tuning on my local machine (works faster than kaggle if you have a good CPU). I wish this could also be helpful and bring some convenience for others.

Please refer to the original notebook if you want more data analysis and visualization.

## first things we need to do...

In [ ]:
import pandas as pd
import numpy as np

import os
import random
import re
import math
import time

# modules forxgboost modeling
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, cross_validate
from sklearn.metrics import roc_auc_score, roc_curve

# ignore warnings for clearner ouputs
import warnings
warnings.filterwarnings('ignore')

In [ ]:
seed = 42
random.seed(42)
np.random.seed(42)

In [ ]:
# setting path, change path here for other melanoma dataset

base_path = '/kaggle/input/siim-isic-melanoma-classification'
train_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_img_path = '/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
img_stats_path = '/kaggle/input/melanoma2020imgtabular'

## load data and get an impression

In [ ]:
train = pd.read_csv(os.path.join(base_path, 'train.csv'))
test = pd.read_csv(os.path.join(base_path, 'test.csv'))
sample = pd.read_csv(os.path.join(base_path, 'sample_submission.csv'))

In [ ]:
f'train features: {train.columns.tolist()}', f'test_featuers: {test.columns.tolist()};'

In [ ]:
# do some simple renaming for complecatec feature name

train.columns = [
    'img_name', 'id', 'sex', 'age', 'location', 'diagnosis',
    'benign_malignant', 'target'
]
test.columns = ['img_name', 'id', 'sex', 'age', 'location']

In [ ]:
train.head()

In [ ]:
test.tail()

# data preprocessing

## Step1: fill missing value

In [ ]:
# now I'm using my own code for simplicity
def fill_missing(df):
    '''The strategy here is:
    1. For cat features we fill 'unknown' if catgories are many else mode
    2. For con features we fill the median
    '''
    df = df.copy()
    # fill nan values
    df.location.fillna(value='unknown', inplace=True)
    df.sex.fillna(value=train.sex.mode()[0], inplace=True)
    df.age.fillna(value=df.age.median(), inplace=True)

    return df

In [ ]:
train = fill_missing(train)
test = fill_missing(test)

In [ ]:
train.isnull().any(), test.isnull().any()

Thanks to this great dataset by Marcelo Kittlein [here](https://www.kaggle.com/kittlein/landscape)

## Step2: feature engineering - concat landscape attributes from images

In [ ]:
# Loading lanscape data

train40 = pd.read_csv('../input/melanoma2020imgtabular/train40Features.csv')
test40 = pd.read_csv('../input/melanoma2020imgtabular/test40Features.csv')

trainmet = pd.read_csv('../input/melanoma2020imgtabular/trainMetrics.csv')
testmet = pd.read_csv('../input/melanoma2020imgtabular/testMetrics.csv')

In [ ]:
# drop duplicate data from landscape dataset
train40.drop(['sex', 'age_approx', 'anatom_site_general_challenge'],
             axis=1,
             inplace=True)
test40.drop(['sex', 'age_approx', 'anatom_site_general_challenge'],
            axis=1,
            inplace=True)

# merging both datasets
train = pd.concat([train, train40, trainmet], axis=1)
test = pd.concat([test, test40, testmet], axis=1)

In [ ]:
train.head()

## Step3: label encoding

In [ ]:
# def label_encoding(df):
#     df = df.copy()
#     # encode labels
#     location = LabelEncoder()
#     sex = LabelEncoder()
#     location_data = location.fit_transform(df.location)
#     sex_data = sex.fit_transform(df.sex)
#     df.location = location_data
#     df.sex = sex_data
    
#     return df

# train = label_encoding(train)
# test = label_encoding(test)

In [ ]:
def dummy_encoding(df):
    df = df.copy()
    
    # dummy encoding for label sex and location
    sex_dummies = pd.get_dummies(df.sex, prefix='sex')
    location_dummies = pd.get_dummies(df.location, prefix='location')
    df = pd.concat([df, sex_dummies], axis=1)
    df = pd.concat([df, location_dummies], axis=1)
    
    return df

In [ ]:
train = dummy_encoding(train)
test = dummy_encoding(test)

In [ ]:
train.head()

## Step4: feature cleaning - drop uneccesary features which are not helpful after all feature engineering

In [ ]:
train.drop(['sex','img_name','id','diagnosis','benign_malignant', 'location'], axis=1, inplace=True)
test.drop(['sex','img_name','id', 'location'], axis=1, inplace=True)

In [ ]:
train.head()

looks like we've got what we want

# XGBoost modelling

Here I only use the simplified meta prediction from the original notebook as I already know the problem of overfitting and the need of more randomness. Feeling really not neccessary to the analyze and do the testing again.

Also I found the dropping process is no longer needed here as the dataset of landscape has changed  so the train/test split is no longer that obvious for our model.

## process well-preprocessed tabular data for modelling

In [ ]:
# input and output for modelling

X = train.drop('target', axis=1)
y = train.target

In [ ]:
# 5 stratified KFold with holdout validating

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=seed)
cv = StratifiedKFold(5, shuffle=True, random_state=seed)

In [ ]:
X_train.head()

In [ ]:
# drop features for overfitting
# X_train.drop(['n_images', 'image_size','width','height','total_pixels','reds','blues','greens','mean_colors', 'age_min', 'age_max'], axis=1, inplace=True)
# test.drop(['n_images', 'image_size','width','height','total_pixels','reds','blues','greens','mean_colors', 'age_min', 'age_max'], axis=1, inplace=True)

## set up model hyperparameters for fine-tuning

same as the original code 

In [ ]:
xg = xgb.XGBClassifier(
    n_estimators=750,
    min_child_weight=0.81,
    learning_rate=0.025,
    max_depth=2,
    subsample=0.80,
    colsample_bytree=0.42,
    gamma=0.10,
    random_state=42,
    n_jobs=-1,
)

In [ ]:
estimators = [xg]

## cross validation scheme

Great code also from the original notebook, thanks!

In [ ]:
def model_check(X_train, y_train, estimators, cv):
    model_table = pd.DataFrame()
    row_index = 0
    
    for est in estimators:
        MLA_name = est.__class__.__name__
        model_table.loc[row_index, 'Model Name'] = MLA_name
        
        cv_results = cross_validate(est,
                                    X_train,
                                    y_train,
                                    cv=cv,
                                    scoring='roc_auc',
                                    return_train_score=True,
                                    n_jobs=-1)
        
        model_table.loc[row_index,
                        'Train roc Mean'] = cv_results['train_score'].mean()
        model_table.loc[row_index,
                        'Test roc Mean'] = cv_results['test_score'].mean()
        model_table.loc[row_index, 'Test Std'] = cv_results['test_score'].std()
        model_table.loc[row_index, 'Time'] = cv_results['fit_time'].mean()
        
        row_index += 1
    
    model_table.sort_values(by=['Test roc Mean'],
                           ascending=False,
                           inplace=True)
    
    return model_table

## make predictions

In [ ]:
raw_models = model_check(X_train, y_train, [xg], cv)
display(raw_models)

In [ ]:
# xgboost predict

xg.fit(X_train, y_train)
predictions = xg.predict_proba(test)[:, 1]

In [ ]:
# create submission file with two meta features required

meta_df = pd.DataFrame(columns=['image_name', 'target'])
meta_df['image_name'] = sample['image_name']
meta_df['target'] = predictions
meta_df.to_csv('xgboost_meta_simplified.csv', header=True, index=False)